# ROL - WaveBot


In [1]:
import autograd.numpy as np
import capytaine as cpy
import matplotlib.pyplot as plt

import wecopttool as wot

## set colorblind-friendly colormap for plots
plt.style.use('tableau-colorblind10')

### Waves

In [2]:
wavefreq = 0.3 # Hz
f1 = wavefreq
nfreq = 10
freq = wot.frequency(f1, nfreq, False)

amplitude = 0.0625 # m
phase = 30 # degrees
wavedir = 0 # degrees

waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

### WEC object

In [3]:
wb = wot.geom.WaveBot()  # use standard dimensions
mesh_size_factor = 0.5 # 1.0 for default, smaller to refine mesh
mesh = wb.mesh(mesh_size_factor)
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")

fb.add_translation_dof(name="Heave")
ndof = fb.nb_dofs

bem_data = wot.run_bem(fb, freq)

name = ["PTO_Heave",]
kinematics = np.eye(ndof)
controller = None
loss = None
pto_impedance = None
pto = wot.pto.PTO(ndof, kinematics, controller, pto_impedance, loss, name)

f_add = {'PTO': pto.force_on_wec}

wec = wot.WEC.from_bem(bem_data, f_add=f_add)

[15:36:35] WARNING  Using the geometric centroid as the center of gravity (COG).

           WARNING  Using the center of gravity (COG) as the rotation center for hydrostatics.

The resolution of the mesh might be insufficient for omega ranging from 9.425 to 18.850.
This warning appears when the largest panel of this mesh has radius > wavelength/8.


           WARNING  Mesh resolution for 12 problems:                                                               
                    The resolution of the mesh might be insufficient for omega ranging from 9.425 to 18.850.       
                    This warning appears when the largest panel of this mesh has radius > wavelength/8.

/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[15:36:39] WARNING  Linear damping for DOF "Heave" has negative or close to zero terms. Shifting up damping terms  
                    [5 7 8 9] to a minimum of 1e-06 N/(m/s)

### Objective function

In [4]:
obj_fun = pto.mechanical_average_power
nstate_opt = 2*nfreq

### Solve

In [5]:
# options = {'maxiter': 200}
scale_x_wec = 1 # 1e1
scale_x_opt = 1 # 1e-3
scale_obj = 1 # 1e-2

results = wec.solve(
    waves,
    obj_fun,
    nstate_opt,
    # optim_options=options,
    scale_x_wec=scale_x_wec,
    scale_x_opt=scale_x_opt,
    scale_obj=scale_obj,
    parameter_file='parameters.xml',
    )

opt_mechanical_average_power = results[0]['f']
print(f'Optimal average mechanical power: {opt_mechanical_average_power} W')

/Users/cmichel/repos/WecOptTool/wecopttool/core.py:712: UserWarning: The `squeeze` kwarg to GroupBy is being removed.Pass .groupby(..., squeeze=False) to disable squeezing, which is the new default, and to silence this warning.
  for realization, wave in waves.groupby('realization'):



  Check primal optimization space vector

********** Begin verification of linear algebra. *********************************************

Commutativity of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Associativity of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Identity element of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Inverse elements of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Identity element of scalar multiplication. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Consistency of scalar multiplication with field multiplication. Consistency error: >>>>>>>>>>> 0.000000000000e+00
Distributivity of scalar multiplication with respect to field addition. Consistency error: >>> 0.000000000000e+00
Distributivity of scalar multiplication with respect to vector 

ValueError: shapes (20,20) and (0,1) not aligned: 20 (dim 1) != 0 (dim 0)

### Analyzing results